In [1]:
!pip install gputil
!pip install psutil
!pip install humanize

  Created wheel for gputil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7411 sha256=db5b80efe5879faa8193a35a82d2d6db4936dd55a3357b85d54bc5b670937fa1
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil


In [2]:
# Import packages
import os,sys,humanize,psutil,GPUtil
import tensorflow as tf

# Define function
def mem_report():
  print("CPU RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ))
  
  GPUs = GPUtil.getGPUs()
  for i, gpu in enumerate(GPUs):
    print('GPU {:d} ... Mem Free: {:.0f}MB / {:.0f}MB | Utilization {:3.0f}%'.format(i, gpu.memoryFree, gpu.memoryTotal, gpu.memoryUtil*100))
    
# Execute function
mem_report()

CPU RAM Free: 12.7 GB
GPU 0 ... Mem Free: 7611MB / 7611MB | Utilization   0%


In [3]:
import numpy as np
import pandas as pd
import datetime
from matplotlib import pyplot as plt
%matplotlib inline

pd.options.display.max_rows = 10
pd.options.display.max_colwidth = 100
pd.options.display.max_columns = 600
from tqdm import tqdm
import gc

from sklearn.linear_model import HuberRegressor
from sklearn.model_selection import cross_val_predict, KFold
from sklearn.decomposition import PCA

In [4]:
np.random.seed=0

In [5]:
def smape(y_true, y_pred):
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2.0
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0.0
    return np.mean(diff)

def smape_mask(y_true, y_pred, threshold):
    denominator = (np.abs(y_true) + np.abs(y_pred)) 
    diff = np.abs(y_true - y_pred) 
    diff[denominator == 0] = 0.0
    
    return diff <= (threshold / 2.0) * denominator

In [7]:
max_size = 184 # number of days in 2015
path='/content/drive/My Drive/GoogleColab/Web_Traffic_Dataset/'
train_all = pd.read_csv(path+'train_2.csv')

In [8]:
# Reduction of dataset (just take 10% of dataset)
from sklearn.model_selection import train_test_split #library

def sampling_dataset(dataset):
# dataset: pandas dataframe for the population
# data_keep: sampled dataframe based on train_test_split (we will keep this for further processing)
# data_discard: sampled datafram that we remove (we will remove this dataset)
# train_size=0.1  : samples 10% of dataset from the popolation and store at variable data_keep
# random_state=42 : to reproduce same result

  data_keep, data_discard=train_test_split(dataset,train_size=0.05, random_state=42)
  print('Total size and sampled size')
  print(dataset.shape,data_keep.shape)
  data_keep.sort_index(inplace=True)
  data_keep.reset_index(drop=True, inplace=True)
  return data_keep

#eg-
train_all=sampling_dataset(train_all)

Total size and sampled size
(145063, 804) (7253, 804)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [9]:
all_page = train_all.Page.copy()
train_key = train_all[['Page']].copy()
all_page = train_all.Page.copy()

In [10]:
def get_date_index(date, train_all=train_all):
    for idx, c in enumerate(train_all.columns):
        if date == c:
            break
    if idx == len(train_all.columns):
        return None
    return idx

In [11]:
train_end = get_date_index('2016-09-10') + 1
test_start = get_date_index('2016-09-13')

In [12]:
train = train_all.iloc[ : , (train_end - max_size) : train_end].copy().astype('float32')
test = train_all.iloc[:, test_start : (63 + test_start)].copy().astype('float32')

train_all = train_all.iloc[:,-(max_size):].astype('float32')

train = train.iloc[:,::-1].copy()
train_all = train_all.iloc[:,::-1].copy()


In [13]:
train_all.head()


,2017-09-10,2017-09-09,2017-09-08,2017-09-07,2017-09-06,2017-09-05,2017-09-04,2017-09-03,2017-09-02,2017-09-01,2017-08-31,2017-08-30,2017-08-29,2017-08-28,2017-08-27,2017-08-26,2017-08-25,2017-08-24,2017-08-23,2017-08-22,2017-08-21,2017-08-20,2017-08-19,2017-08-18,2017-08-17,2017-08-16,2017-08-15,2017-08-14,2017-08-13,2017-08-12,2017-08-11,2017-08-10,2017-08-09,2017-08-08,2017-08-07,2017-08-06,2017-08-05,2017-08-04,2017-08-03,2017-08-02,2017-08-01,2017-07-31,2017-07-30,2017-07-29,2017-07-28,2017-07-27,2017-07-26,2017-07-25,2017-07-24,2017-07-23,2017-07-22,2017-07-21,2017-07-20,2017-07-19,2017-07-18,2017-07-17,2017-07-16,2017-07-15,2017-07-14,2017-07-13,2017-07-12,2017-07-11,2017-07-10,2017-07-09,2017-07-08,2017-07-07,2017-07-06,2017-07-05,2017-07-04,2017-07-03,2017-07-02,2017-07-01,2017-06-30,2017-06-29,2017-06-28,2017-06-27,2017-06-26,2017-06-25,2017-06-24,2017-06-23,2017-06-22,2017-06-21,2017-06-20,2017-06-19,2017-06-18,2017-06-17,2017-06-16,2017-06-15,2017-06-14,2017-06-13,2017-06-12,2017-06-11,2017-06-10,2017-06-09,2017-06-08,2017-06-07,2017-06-06,2017-06-05,2017-06-04,2017-06-03,2017-06-02,2017-06-01,2017-05-31,2017-05-30,2017-05-29,2017-05-28,2017-05-27,2017-05-26,2017-05-25,2017-05-24,2017-05-23,2017-05-22,2017-05-21,2017-05-20,2017-05-19,2017-05-18,2017-05-17,2017-05-16,2017-05-15,2017-05-14,2017-05-13,2017-05-12,2017-05-11,2017-05-10,2017-05-09,2017-05-08,2017-05-07,2017-05-06,2017-05-05,2017-05-04,2017-05-03,2017-05-02,2017-05-01,2017-04-30,2017-04-29,2017-04-28,2017-04-27,2017-04-26,2017-04-25,2017-04-24,2017-04-23,2017-04-22,2017-04-21,2017-04-20,2017-04-19,2017-04-18,2017-04-17,2017-04-16,2017-04-15,2017-04-14,2017-04-13,2017-04-12,2017-04-11,2017-04-10,2017-04-09,2017-04-08,2017-04-07,2017-04-06,2017-04-05,2017-04-04,2017-04-03,2017-04-02,2017-04-01,2017-03-31,2017-03-30,2017-03-29,2017-03-28,2017-03-27,2017-03-26,2017-03-25,2017-03-24,2017-03-23,2017-03-22,2017-03-21,2017-03-20,2017-03-19,2017-03-18,2017-03-17,2017-03-16,2017-03-15,2017-03-14,2017-03-13,2017-03-12,2017-03-11
0,18.0,43.0,29.0,67.0,48.0,34.0,31.0,25.0,33.0,31.0,20.0,18.0,24.0,38.0,34.0,32.0,32.0,94.0,49.0,41.0,28.0,21.0,38.0,48.0,31.0,30.0,26.0,36.0,62.0,43.0,39.0,19.0,34.0,69.0,166.0,101.0,166.0,103.0,21.0,38.0,90.0,113.0,24.0,17.0,21.0,28.0,141.0,35.0,51.0,17.0,12.0,15.0,31.0,33.0,52.0,103.0,21.0,23.0,19.0,23.0,22.0,17.0,28.0,15.0,37.0,41.0,52.0,24.0,28.0,25.0,48.0,39.0,59.0,26.0,23.0,15.0,14.0,26.0,24.0,19.0,24.0,50.0,23.0,21.0,54.0,31.0,25.0,39.0,28.0,36.0,33.0,18.0,28.0,27.0,72.0,23.0,34.0,20.0,33.0,33.0,38.0,29.0,25.0,25.0,41.0,15.0,22.0,22.0,16.0,18.0,23.0,18.0,24.0,32.0,38.0,25.0,30.0,51.0,45.0,21.0,18.0,10.0,20.0,25.0,16.0,14.0,23.0,27.0,30.0,37.0,17.0,24.0,25.0,18.0,35.0,31.0,11.0,28.0,40.0,19.0,20.0,35.0,24.0,35.0,20.0,12.0,27.0,43.0,17.0,18.0,17.0,24.0,27.0,24.0,32.0,30.0,18.0,26.0,63.0,30.0,24.0,36.0,28.0,30.0,38.0,20.0,16.0,9.0,19.0,33.0,25.0,17.0,23.0,32.0,31.0,45.0,70.0,30.0,31.0,41.0,48.0,23.0,44.0,62.0
1,NaN,NaN,0.0,1.0,3.0,NaN,0.0,NaN,NaN,NaN,1.0,NaN,2.0,5.0,7.0,3.0,4.0,6.0,20.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,1.0,0.0,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,3.0,2.0,NaN,NaN,NaN,NaN,1.0,1.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,1.0,NaN,NaN,1.0,NaN,NaN,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
2,2.0,14.0,7.0,15.0,23.0,12.0,22.0,15.0,12.0,17.0,31.0,15.0,16.0,25.0,20.0,22.0,47.0,34.0,35.0,6.0,7.0,12.0,6.0,13.0,8.0,10.0,7.0,13.0,32.0,9.0,11.0,6.0,9.0,15.0,22.0,28.0,29.0,9.0,12.0,40.0,10.0,22.0,14.0,36.0,11.0,11.0,16.0,31.0,7.0,22.0,5.0,13.0,32.0,43.0,22.0,23.0,4.0,12.0,8.0,11.0,15.0,7.0,13.0,25.0,24

In [14]:
train.head()

,2016-09-10,2016-09-09,2016-09-08,2016-09-07,2016-09-06,2016-09-05,2016-09-04,2016-09-03,2016-09-02,2016-09-01,2016-08-31,2016-08-30,2016-08-29,2016-08-28,2016-08-27,2016-08-26,2016-08-25,2016-08-24,2016-08-23,2016-08-22,2016-08-21,2016-08-20,2016-08-19,2016-08-18,2016-08-17,2016-08-16,2016-08-15,2016-08-14,2016-08-13,2016-08-12,2016-08-11,2016-08-10,2016-08-09,2016-08-08,2016-08-07,2016-08-06,2016-08-05,2016-08-04,2016-08-03,2016-08-02,2016-08-01,2016-07-31,2016-07-30,2016-07-29,2016-07-28,2016-07-27,2016-07-26,2016-07-25,2016-07-24,2016-07-23,2016-07-22,2016-07-21,2016-07-20,2016-07-19,2016-07-18,2016-07-17,2016-07-16,2016-07-15,2016-07-14,2016-07-13,2016-07-12,2016-07-11,2016-07-10,2016-07-09,2016-07-08,2016-07-07,2016-07-06,2016-07-05,2016-07-04,2016-07-03,2016-07-02,2016-07-01,2016-06-30,2016-06-29,2016-06-28,2016-06-27,2016-06-26,2016-06-25,2016-06-24,2016-06-23,2016-06-22,2016-06-21,2016-06-20,2016-06-19,2016-06-18,2016-06-17,2016-06-16,2016-06-15,2016-06-14,2016-06-13,2016-06-12,2016-06-11,2016-06-10,2016-06-09,2016-06-08,2016-06-07,2016-06-06,2016-06-05,2016-06-04,2016-06-03,2016-06-02,2016-06-01,2016-05-31,2016-05-30,2016-05-29,2016-05-28,2016-05-27,2016-05-26,2016-05-25,2016-05-24,2016-05-23,2016-05-22,2016-05-21,2016-05-20,2016-05-19,2016-05-18,2016-05-17,2016-05-16,2016-05-15,2016-05-14,2016-05-13,2016-05-12,2016-05-11,2016-05-10,2016-05-09,2016-05-08,2016-05-07,2016-05-06,2016-05-05,2016-05-04,2016-05-03,2016-05-02,2016-05-01,2016-04-30,2016-04-29,2016-04-28,2016-04-27,2016-04-26,2016-04-25,2016-04-24,2016-04-23,2016-04-22,2016-04-21,2016-04-20,2016-04-19,2016-04-18,2016-04-17,2016-04-16,2016-04-15,2016-04-14,2016-04-13,2016-04-12,2016-04-11,2016-04-10,2016-04-09,2016-04-08,2016-04-07,2016-04-06,2016-04-05,2016-04-04,2016-04-03,2016-04-02,2016-04-01,2016-03-31,2016-03-30,2016-03-29,2016-03-28,2016-03-27,2016-03-26,2016-03-25,2016-03-24,2016-03-23,2016-03-22,2016-03-21,2016-03-20,2016-03-19,2016-03-18,2016-03-17,2016-03-16,2016-03-15,2016-03-14,2016-03-13,2016-03-12,2016-03-11
0,14.0,16.0,20.0,15.0,28.0,25.0,27.0,26.0,21.0,25.0,23.0,29.0,20.0,29.0,20.0,26.0,30.0,45.0,38.0,11.0,15.0,25.0,22.0,17.0,12.0,22.0,24.0,26.0,37.0,30.0,31.0,32.0,23.0,28.0,38.0,23.0,15.0,75.0,20.0,22.0,32.0,26.0,21.0,20.0,18.0,30.0,32.0,16.0,31.0,33.0,31.0,29.0,120.0,28.0,28.0,25.0,18.0,20.0,37.0,19.0,36.0,25.0,29.0,26.0,30.0,25.0,24.0,24.0,71.0,25.0,18.0,23.0,30.0,25.0,21.0,20.0,20.0,22.0,28.0,20.0,24.0,32.0,21.0,18.0,26.0,34.0,38.0,28.0,23.0,35.0,23.0,19.0,21.0,19.0,13.0,16.0,20.0,14.0,22.0,17.0,19.0,24.0,16.0,25.0,21.0,18.0,16.0,20.0,13.0,12.0,20.0,41.0,19.0,26.0,16.0,10.0,20.0,16.0,17.0,17.0,54.0,22.0,29.0,23.0,17.0,21.0,16.0,10.0,22.0,13.0,21.0,23.0,22.0,26.0,14.0,18.0,15.0,17.0,35.0,17.0,38.0,27.0,9.0,9.0,11.0,21.0,28.0,30.0,38.0,32.0,32.0,37.0,46.0,40.0,39.0,44.0,41.0,38.0,38.0,33.0,32.0,8.0,29.0,13.0,20.0,17.0,16.0,27.0,10.0,24.0,13.0,15.0,10.0,168.0,24.0,15.0,15.0,8.0,13.0,16.0,37.0,13.0,22.0,8.0
1,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,1.0,0.0,0.0,NaN
2,7.0,5.0,1.0,25.0,53.0,15.0,16.0,6.0,7.0,108.0,133.0,8.0,28.0,14.0,27.0,3.0,4.0,7.0,4.0,2.0,17.0,6.0,43.0,15.0,20.0,2.0,17.0,41.0,10.0,4.0,8.0,18.0,5.0,12.0,4.0,2.0,12.0,7.0,7.0,1.0,4.0,9.0,52.0,67.0,7.0,23.0,8.0,6.0,23.0,11.0,6.0,18.0,7.0,4.0,10.0,6.0,24.0,54.0,4.0,16.0,7.0,17.0,8.0,37.0,1.0,35.0,24.0,16.0,15.0,6.0

In [15]:
test.head()


,2016-09-13,2016-09-14,2016-09-15,2016-09-16,2016-09-17,2016-09-18,2016-09-19,2016-09-20,2016-09-21,2016-09-22,2016-09-23,2016-09-24,2016-09-25,2016-09-26,2016-09-27,2016-09-28,2016-09-29,2016-09-30,2016-10-01,2016-10-02,2016-10-03,2016-10-04,2016-10-05,2016-10-06,2016-10-07,2016-10-08,2016-10-09,2016-10-10,2016-10-11,2016-10-12,2016-10-13,2016-10-14,2016-10-15,2016-10-16,2016-10-17,2016-10-18,2016-10-19,2016-10-20,2016-10-21,2016-10-22,2016-10-23,2016-10-24,2016-10-25,2016-10-26,2016-10-27,2016-10-28,2016-10-29,2016-10-30,2016-10-31,2016-11-01,2016-11-02,2016-11-03,2016-11-04,2016-11-05,2016-11-06,2016-11-07,2016-11-08,2016-11-09,2016-11-10,2016-11-11,2016-11-12,2016-11-13,2016-11-14
0,19.0,25.0,48.0,27.0,19.0,15.0,46.0,20.0,21.0,30.0,18.0,29.0,33.0,14.0,23.0,28.0,21.0,41.0,28.0,21.0,39.0,24.0,23.0,20.0,23.0,25.0,25.0,36.0,19.0,23.0,24.0,15.0,21.0,22.0,34.0,21.0,27.0,21.0,21.0,16.0,30.0,31.0,21.0,27.0,18.0,21.0,66.0,24.0,23.0,25.0,23.0,31.0,26.0,17.0,26.0,24.0,23.0,22.0,25.0,13.0,61.0,25.0,41.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,21.0,4.0,41.0,73.0,9.0,2.0,19.0,5.0,60.0,5.0,8.0,4.0,17.0,4.0,3.0,3.0,1.0,9.0,140.0,104.0,104.0,40.0,103.0,50.0,36.0,70.0,44.0,23.0,17.0,20.0,51.0,56.0,76.0,21.0,26.0,14.0,10.0,13.0,6.0,8.0,48.0,59.0,23.0,15.0,12.0,12.0,18.0,25.0,44.0,24.0,29.0,5.0,29.0,6.0,9.0,19.0,10.0,10.0,35.0,1.0,7.0,6.0,93.0
3,6.0,23.0,30.0,43.0,39.0,9.0,56.0,12.0,7.0,22.0,8.0,10.0,45.0,43.0,20.0,9.0,12.0,46.0,47.0,29.0,11.0,33.0,12.0,19.0,15.0,21.0,66.0,22.0,17.0,13.0,63.0,31.0,84.0,26.0,45.0,17.0,24.0,23.0,297.0,68.0,24.0,10.0,14.0,37.0,19.0,32.0,74.0,28.0,21.0,22.0,34.0,14.0,28.0,34.0,15.0,16.0,41.0,15.0,10.0,14.0,68.0,16.0,6.0
4,24.0,14.0,14.0,16.0,20.0,12.0,16.0,14.0,12.0,15.0,18.0,13.0,16.0,19.0,31.0,26.0,18.0,16.0,9.0,50.0,24.0,10.0,16.0,19.0,26.0,11.0,74.0,40.0,15.0,15.0,19.0,21.0,12.0,20.0,19.0,10.0,8.0,19.0,12.0,13.0,18.0,7.0,20.0,25.0,15.0,13.0,14.0,36.0,19.0,22.0,21.0,14.0,10.0,14.0,16.0,24.0,12.0,8.0,16.0,7.0,372.0,14.0,17.0


In [16]:
data = [page.split('_') for page in tqdm(train_key.Page)]

access = ['_'.join(page[-2:]) for page in data]

site = [page[-3] for page in data]

page = ['_'.join(page[:-3]) for page in data]
page[:2]

train_key['PageTitle'] = page
train_key['Site'] = site
train_key['AccessAgent'] = access
train_key.head()

100%|██████████| 7253/7253 [00:00<00:00, 511565.86it/s]


,Page,PageTitle,Site,AccessAgent
0,Beyond_zh.wikipedia.org_all-access_spider,Beyond,zh.wikipedia.org,all-access_spider
1,File:Sebastiaosalgado2006.jpg_zh.wikipedia.org_all-access_spider,File:Sebastiaosalgado2006.jpg,zh.wikipedia.org,all-access_spider
2,GNZ48_zh.wikipedia.org_all-access_spider,GNZ48,zh.wikipedia.org,all-access_spider
3,Gu9udan_zh.wikipedia.org_all-access_spider,Gu9udan,zh.wikipedia.org,all-access_spider
4,IKON_zh.wikipedia.org_all-access_spider,IKON,zh.wikipedia.org,all-access_spider


In [17]:
train_norm = np.log1p(train)
train_norm.head()

,2016-09-10,2016-09-09,2016-09-08,2016-09-07,2016-09-06,2016-09-05,2016-09-04,2016-09-03,2016-09-02,2016-09-01,2016-08-31,2016-08-30,2016-08-29,2016-08-28,2016-08-27,2016-08-26,2016-08-25,2016-08-24,2016-08-23,2016-08-22,2016-08-21,2016-08-20,2016-08-19,2016-08-18,2016-08-17,2016-08-16,2016-08-15,2016-08-14,2016-08-13,2016-08-12,2016-08-11,2016-08-10,2016-08-09,2016-08-08,2016-08-07,2016-08-06,2016-08-05,2016-08-04,2016-08-03,2016-08-02,2016-08-01,2016-07-31,2016-07-30,2016-07-29,2016-07-28,2016-07-27,2016-07-26,2016-07-25,2016-07-24,2016-07-23,2016-07-22,2016-07-21,2016-07-20,2016-07-19,2016-07-18,2016-07-17,2016-07-16,2016-07-15,2016-07-14,2016-07-13,2016-07-12,2016-07-11,2016-07-10,2016-07-09,2016-07-08,2016-07-07,2016-07-06,2016-07-05,2016-07-04,2016-07-03,2016-07-02,2016-07-01,2016-06-30,2016-06-29,2016-06-28,2016-06-27,2016-06-26,2016-06-25,2016-06-24,2016-06-23,2016-06-22,2016-06-21,2016-06-20,2016-06-19,2016-06-18,2016-06-17,2016-06-16,2016-06-15,2016-06-14,2016-06-13,2016-06-12,2016-06-11,2016-06-10,2016-06-09,2016-06-08,2016-06-07,2016-06-06,2016-06-05,2016-06-04,2016-06-03,2016-06-02,2016-06-01,2016-05-31,2016-05-30,2016-05-29,2016-05-28,2016-05-27,2016-05-26,2016-05-25,2016-05-24,2016-05-23,2016-05-22,2016-05-21,2016-05-20,2016-05-19,2016-05-18,2016-05-17,2016-05-16,2016-05-15,2016-05-14,2016-05-13,2016-05-12,2016-05-11,2016-05-10,2016-05-09,2016-05-08,2016-05-07,2016-05-06,2016-05-05,2016-05-04,2016-05-03,2016-05-02,2016-05-01,2016-04-30,2016-04-29,2016-04-28,2016-04-27,2016-04-26,2016-04-25,2016-04-24,2016-04-23,2016-04-22,2016-04-21,2016-04-20,2016-04-19,2016-04-18,2016-04-17,2016-04-16,2016-04-15,2016-04-14,2016-04-13,2016-04-12,2016-04-11,2016-04-10,2016-04-09,2016-04-08,2016-04-07,2016-04-06,2016-04-05,2016-04-04,2016-04-03,2016-04-02,2016-04-01,2016-03-31,2016-03-30,2016-03-29,2016-03-28,2016-03-27,2016-03-26,2016-03-25,2016-03-24,2016-03-23,2016-03-22,2016-03-21,2016-03-20,2016-03-19,2016-03-18,2016-03-17,2016-03-16,2016-03-15,2016-03-14,2016-03-13,2016-03-12,2016-03-11
0,2.708050,2.833213,3.044523,2.772589,3.367296,3.258096,3.332205,3.295837,3.091043,3.258096,3.178054,3.401197,3.044523,3.401197,3.044523,3.295837,3.433987,3.828641,3.663562,2.484907,2.772589,3.258096,3.135494,2.890372,2.564949,3.135494,3.218876,3.295837,3.637586,3.433987,3.465736,3.496508,3.178054,3.367296,3.663562,3.178054,2.772589,4.330733,3.044523,3.135494,3.496508,3.295837,3.091043,3.044523,2.944439,3.433987,3.496508,2.833213,3.465736,3.526361,3.465736,3.401197,4.795791,3.367296,3.367296,3.258096,2.944439,3.044523,3.637586,2.995732,3.610918,3.258096,3.401197,3.295837,3.433987,3.258096,3.218876,3.218876,4.276666,3.258096,2.944439,3.178054,3.433987,3.258096,3.091043,3.044523,3.044523,3.135494,3.367296,3.044523,3.218876,3.496508,3.091043,2.944439,3.295837,3.555348,3.663562,3.367296,3.178054,3.583519,3.178054,2.995732,3.091043,2.995732,2.639057,2.833213,3.044523,2.708050,3.135494,2.890372,2.995732,3.218876,2.833213,3.258096,3.091043,2.944439,2.833213,3.044523,2.639057,2.564949,3.044523,3.737670,2.995732,3.295837,2.833213,2.397895,3.044523,2.833213,2.890372,2.890372,4.007333,3.135494,3.401197,3.178054,2.890372,3.091043,2.833213,2.397895,3.135494,2.639057,3.091043,3.178054,3.135494,3.295837,2.708050,2.944439,2.772589,2.890372,3.583519,2.890372,3.663562,3.332205,2.302585,2.302585,2.484907,3.091043,3.367296,3.433987,3.663562,3.496508,3.496508,3.637586,3.850148,3.713572,3.688879,3.806663,3.737670,3.663562,3.663562,3.526361,3.496508,2.197225,3.401197,2.639057,3.044523,2.890372,2.833213,3.332205,2.397895,3.218876,2.639057,2.772589,2.397895,5.129899,3.218876,2.772589,2.772589,2.197225,2.639057,2.833213,3.637586,2.639057,3.135494,2.197225
1,NaN,NaN,0.693147,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.693147,NaN,0.693147,NaN,0.693147,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.693147,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.693147,0.693147,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.693147,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.693147,NaN,1.098612,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0

In [18]:
train_all_norm = np.log1p(train_all)

In [19]:
test['Page'] = all_page
test = pd.melt(test, id_vars=['Page'], var_name='Date', value_name='Visits_true')
test['Week'] = pd.to_datetime(test.Date).dt.dayofyear // 7
test = test.merge(train_key, how='left', on='Page')
test['Visits_true'] = test.Visits_true.astype('float32')
test['Visits_norm'] = np.log1p(test.Visits_true).astype('float32')
test = test[test.Visits_true.isnull() != True].reset_index(drop=True)

test.head()

,Page,Date,Visits_true,Week,PageTitle,Site,AccessAgent,Visits_norm
0,Beyond_zh.wikipedia.org_all-access_spider,2016-09-13,19.0,36,Beyond,zh.wikipedia.org,all-access_spider,2.995732
1,GNZ48_zh.wikipedia.org_all-access_spider,2016-09-13,21.0,36,GNZ48,zh.wikipedia.org,all-access_spider,3.091043
2,Gu9udan_zh.wikipedia.org_all-access_spider,2016-09-13,6.0,36,Gu9udan,zh.wikipedia.org,all-access_spider,1.945910
3,IKON_zh.wikipedia.org_all-access_spider,2016-09-13,24.0,36,IKON,zh.wikipedia.org,all-access_spider,3.218876
4,Rewrite_zh.wikipedia.org_all-access_spider,2016-09-13,41.0,36,Rewrite,zh.wikipedia.org,all-access_spider,3.737670


In [20]:
test_all = pd.read_csv(path+'key_2.csv')
test_all.head()

,Page,Id
0,007_スペクター_ja.wikipedia.org_all-access_all-agents_2017-09-13,0b293039387a
1,007_スペクター_ja.wikipedia.org_all-access_all-agents_2017-09-14,7114389dd824
2,007_スペクター_ja.wikipedia.org_all-access_all-agents_2017-09-15,057b02ff1f09
3,007_スペクター_ja.wikipedia.org_all-access_all-agents_2017-09-16,bd2aca21caa3
4,007_スペクター_ja.wikipedia.org_all-access_all-agents_2017-09-17,c0effb42cdd5


In [21]:
test_all['Date'] = [page[-10:] for page in tqdm(test_all.Page)]
test_all['Page'] = [page[:-11] for page in tqdm(test_all.Page)]
test_all.head()

100%|██████████| 8993906/8993906 [00:04<00:00, 2237962.73it/s]


,Page,Id,Date
0,007_スペクター_ja.wikipedia.org_all-access_all-agents,0b293039387a,2017-09-13
1,007_スペクター_ja.wikipedia.org_all-access_all-agents,7114389dd824,2017-09-14
2,007_スペクター_ja.wikipedia.org_all-access_all-agents,057b02ff1f09,2017-09-15
3,007_スペクター_ja.wikipedia.org_all-access_all-agents,bd2aca21caa3,2017-09-16
4,007_スペクター_ja.wikipedia.org_all-access_all-agents,c0effb42cdd5,2017-09-17


In [22]:
test_all['Week'] = pd.to_datetime(test_all.Date).dt.dayofyear // 7
test_all = test_all.merge(train_key, how='left', on='Page')
test_all.head()

,Page,Id,Date,Week,PageTitle,Site,AccessAgent
0,007_スペクター_ja.wikipedia.org_all-access_all-agents,0b293039387a,2017-09-13,36,NaN,NaN,NaN
1,007_スペクター_ja.wikipedia.org_all-access_all-agents,7114389dd824,2017-09-14,36,NaN,NaN,NaN
2,007_スペクター_ja.wikipedia.org_all-access_all-agents,057b02ff1f09,2017-09-15,36,NaN,NaN,NaN
3,007_スペクター_ja.wikipedia.org_all-access_all-agents,bd2aca21caa3,2017-09-16,37,NaN,NaN,NaN
4,007_スペクター_ja.wikipedia.org_all-access_all-agents,c0effb42cdd5,2017-09-17,37,NaN,NaN,NaN


In [23]:
sites = train_key.Site.unique()
sites


array(['zh.wikipedia.org', 'fr.wikipedia.org', 'en.wikipedia.org',
       'commons.wikimedia.org', 'ru.wikipedia.org', 'www.mediawiki.org',
       'de.wikipedia.org', 'ja.wikipedia.org', 'es.wikipedia.org'],
      dtype=object)

In [24]:

# all visits is median

def add_median(test, train, train_key, periods, max_periods, first_train_weekday):
    train =  train.iloc[:,:7*max_periods]
    train_weekday = np.array([(first_train_weekday-w) % 7 for w in range(train.shape[1])])
    train_week_idx = [i for i,w in enumerate(train_weekday) if w <= 4]
    train_week = train.iloc[:,train_week_idx]
    train_weekend_idx = [i for i,w in enumerate(train_weekday) if w > 4]
    train_weekend = train.iloc[:,train_weekend_idx]

    test_week = (test.WeekDay <= 4)
    test_weekend = ~test_week
    test['WeekEnd'] = 1 * test_weekend
    df = train_key[['Page']].copy()
    df['AllVisits'] = train.median(axis=1).fillna(0)
    test = test.merge(df, how='left', on='Page', copy=False)
    test.AllVisits = test.AllVisits.fillna(0).astype('float32')
    
    for (w1, w2) in tqdm(periods):
        
        df = train_key[['Page']].copy()
        c = 'median_%d_%d' % (w1, w2)
        df[c] = train.iloc[:,7*w1:7*w2].median(axis=1, skipna=True) 
        test = test.merge(df, how='left', on='Page', copy=False)
        test[c] = (test[c] - test.AllVisits).fillna(0).astype('float32')
        
        c = 'median_day_%d_%d' % (w1, w2)
        test_page = test[['Page']].copy()
        
        df = train_key[['Page']].copy()
        df[c] = train_week.iloc[:,5*w1:5*w2].median(axis=1, skipna=True) 
        df = test_page.loc[test_week].merge(df, how='left', on='Page', copy=False)
        test.loc[test_week, c] = df[c].values
        
        df = train_key[['Page']].copy()
        df[c] = train_weekend.iloc[:,2*w1:2*w2].median(axis=1, skipna=True) 
        df = test_page.loc[test_weekend].merge(df, how='left', on='Page', copy=False)
        test.loc[test_weekend, c] = df[c].values

        test[c] = (test[c] - test.AllVisits).fillna(0).astype('float32')

    gc.collect()

    return test

In [25]:
test0 = test.copy()
test_all0 = test_all.copy()

In [28]:

npca = 0
max_periods = 20
periods = [(0,1), (1,2), (2,3), (3,4), 
           (4,5), (5,6), (6,7), (7,8),
           (0,2), (0,4),
           (0,8), (0,12), 
           (0,16),
           (0, max_periods)
          ]

test, test_all = test0.copy(), test_all0.copy()

res = 0
res_den = 0
out = []

test['Visits'] = 0
test_all['Visits'] = 0

threshold = 1.25

for site in sites:
    print(site)

    train_norm_site = train_norm[train_key.Site == site]
    train_all_norm_site = train_all_norm[train_key.Site == site]
    train_key_site = train_key[train_key.Site == site]

    test_site = test[test.Site == site].reset_index(drop=True)
    test_site['Date'] = pd.to_datetime(test_site.Date)
    test_site['WeekDay'] = test_site.Date.dt.dayofweek
    
    test_all_site = test_all[test_all.Site == site].reset_index(drop=True)
    test_all_site['Date'] = pd.to_datetime(test_all_site.Date)
    test_all_site['WeekDay'] = test_all_site.Date.dt.dayofweek

    test1 = add_median(test_site, train_norm_site, train_key_site, periods, max_periods, 3)
    test_all1 = add_median(test_all_site, train_all_norm_site, train_key_site, periods, max_periods, 5)
    
    test1.Visits_norm -= test1.AllVisits
    
    kf = KFold(5, shuffle=False)
    num_cols = (['median_day_%d_%d' % (w1,w2) for (w1,w2) in periods]) 

    print('threshold: %0.2f' % threshold)
    res_site = 0
    res_site_den = 0
    for week in test_site.Week.unique():
        #print('week:', week)
        test2 = test1[test1.Week == week].reset_index(drop=True)
        
        lr = HuberRegressor(epsilon=1)
        
        lr.fit(test2[num_cols], test2.Visits_norm)
        y = lr.predict(test2[num_cols])
        y += test2.AllVisits
        y = np.expm1(y)
        y[y < 0.85] = 0
        res_site_week0 = smape(test2.Visits_true, y)
        # print(site, week, 'smape: %0.5f' % res_site_week0)
        
        if site not in ['commons.wikimedia.org', 'www.mediawiki.org',]:
        
            mask = smape_mask(test2.Visits_true, y, threshold)
            test3 = test2[mask]
            lr.fit(test3[num_cols], test3.Visits_norm)
            y = lr.predict(test2[num_cols])
            y += test2.AllVisits
            y = np.expm1(y)
            y[y < 0.85] = 0
            res_site_week = smape(test2.Visits_true, y)
            #print(site, week, 'smape: %0.5f' % res_site_week)
        else:
            res_site_week = res_site_week0
            
        print(site, week, 'smape: %0.5f' % res_site_week, 'delta: %0.5f' % (res_site_week0 - res_site_week))
        test.loc[(test.Site == site) & (test.Week == week), 'Visits'] = y
        
        
        res_site += res_site_week * test2.shape[0]
        res_site_den += test2.shape[0]
        res += res_site_week * test2.shape[0]
        res_den += test2.shape[0]
        out.append((test2.Visits_true.values, y, test2.shape[0]))
        
        test_all2 = test_all1[test_all1.Week == week]
        y = lr.predict(test_all2[num_cols])
        y += test_all2.AllVisits
        y = np.expm1(y)
        y[y < 0.85] = 0
        test_all.loc[(test_all.Site == site) & (test_all.Week == week), 'Visits'] = y

    res_site /= res_site_den
    print('smape %s: %0.5f' % (site, res_site))

        
res /= res_den
print('smape all: %0.5f' % res)

y_true = np.concatenate([y_true for (y_true, y_pred, size) in out], axis=0)
y_pred = np.concatenate([y_pred for (y_true, y_pred, size) in out], axis=0)
print('smape all: %0.5f' % smape(y_true, y_pred))


zh.wikipedia.org


100%|██████████| 14/14 [00:00<00:00, 17.34it/s]


threshold: 1.25
zh.wikipedia.org 36 smape: 0.33574 delta: -0.00006
zh.wikipedia.org 37 smape: 0.38991 delta: 0.00040


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_huber.py:296: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


zh.wikipedia.org 38 smape: 0.42934 delta: 0.00028


...

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_huber.py:296: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_huber.py:296: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


es.wikipedia.org 43 smape: 0.41695 delta: 0.00085
es.wikipedia.org 44 smape: 0.43859 delta: 0.00047


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_huber.py:296: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_huber.py:296: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


es.wikipedia.org 45 smape: 0.44078 delta: 0.00074
smape es.wikipedia.org: 0.39427
smape all: 0.45283
smape all: 0.45283


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


In [29]:
test.Visits = test.Visits.round(3)
test_all.Visits = test_all.Visits.round(3)

test[['Page', 'Date', 'Visits']].to_csv(path+'Perd_part2_pred_10_stage2_sept_10_train.csv', index=False)
test_all[['Id', 'Visits']].to_csv(path+'Perd_part2_pred_10_stage2_sept_10_test.csv', index=False)

In [30]:
test.head()

,Page,Date,Visits_true,Week,PageTitle,Site,AccessAgent,Visits_norm,Visits
0,Beyond_zh.wikipedia.org_all-access_spider,2016-09-13,19.0,36,Beyond,zh.wikipedia.org,all-access_spider,2.995732,16.616
1,GNZ48_zh.wikipedia.org_all-access_spider,2016-09-13,21.0,36,GNZ48,zh.wikipedia.org,all-access_spider,3.091043,7.874
2,Gu9udan_zh.wikipedia.org_all-access_spider,2016-09-13,6.0,36,Gu9udan,zh.wikipedia.org,all-access_spider,1.945910,14.323
3,IKON_zh.wikipedia.org_all-access_spider,2016-09-13,24.0,36,IKON,zh.wikipedia.org,all-access_spider,3.218876,13.675
4,Rewrite_zh.wikipedia.org_all-access_spider,2016-09-13,41.0,36,Rewrite,zh.wikipedia.org,all-access_spider,3.737670,13.420


In [31]:
test.Visits.mean()

1642.134271459227

In [32]:

test_all.Visits.mean()

0.15783072799829476